# <span style="color:red">Solutions to Exam 1<span>

## Financial Analytics

### UChicago ADSP

#### Spring 2024
* Mark Hendricks
* hendricks@uchicago.edu

***

# Instructions

## Please note the following:

Time
* You have 90 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.

Submission
* You will upload your solution to the Exam "assignment" on Canvas.
* Your submission should be readable, (the graders can understand your answers,) and it should include all code used in your analysis in a file format that the code can be executed. (ie. .ipynb preferred, .pdf is unacceptable.)

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation, (only for this exam.)

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:

#### Section 2
* New file: `select_equity_returns.xlsx`

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 35     |
| 2       | 25     |
| 3       | 25     |
| **Total**   | **85**|

### Each numbered question is worth 5 points unless otherwise specified.

***

# 1. Long-Short Bond Trade

Consider the following market data as of `Dec 29, 2023`.

The table below shows two Treasury securities, a T-note and a T-bond. They mature on the same date.

In [1]:
import pandas as pd
summary = pd.DataFrame(index=[],columns = [207391,204095],dtype=float)
summary.loc['issue date'] = ['2019-08-15','1999-08-15']
summary.loc['maturity date'] = ['2029-08-15','2029-08-15']
summary.loc['coupon rate'] = [.01625, .06125]
summary.loc['clean price'] = [89.03125,111.0391]
summary.loc['accrued interest'] = [.6005, 2.2636]
summary.loc['ytm'] = [.037677, .038784]
summary.loc['duration'] = [5.3494,4.7967]
summary

,207391,204095
issue date,2019-08-15,1999-08-15
maturity date,2029-08-15,2029-08-15
coupon rate,0.01625,0.06125
clean price,89.03125,111.0391
accrued interest,0.6005,2.2636
ytm,0.037677,0.038784
duration,5.3494,4.7967


### 1.1.

Explain the long-short trade you would enter based on the market data above, without any further calculation. 

### 1.2.

Size your trade.
* Suppose the long side is set to $100 million market value. 
* Size the short to be duration-neutral.

Report the market value in the short-side of the trade and the number of long and short contracts.

Be sure to account for the **dirty** price.

### 1.3.

What are the risks of this trade in the short-term and in the long-term?

### 1.4.

Use **modified duration**--not the Macauley duration reported above--to estimate how much pnl will be earned if the securities converge (symmetrically).

### 1.5.

For which of these securities will the duration approximation be less accurate? Explain.

### 1.6.

Is the difference in YTM consistent with our discussion of a liquidity premium in bonds? Explain.

### 1.7.

Assume, just for this part of the problem...
* current date is Feb 15, 2024, so both mature in exaclty 5.5 years.
* the coupon has just been paid, so the accrued interest is zero
* the price of both bonds are 91.02 and 109.5, respectively. (These are made up, don't try to rationalize them.)

Calculate the YTM of each bond.

***

# <span style="color:red">Solution 1</span>

In [2]:
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.optimize import minimize

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)

import sys
sys.path.insert(0, '../cmds')
from portfolio import get_ols_metrics
from treasury_cmds import *
from tradebondpair import *

## <span style="color:red">1.1</span>

We should go **long** the **higher** YTM, and **short** the **lower** YTM.

Note that this is **not** the same as going long the bond with lower price and shorting the bond with higher price. Given the difference in coupons, the higher price is not indicative of the 30-year being a worse deal. Rather, its higher YTM shows it is more attractive.

Optionally, one could mention that the long-short trade might be sized to 
* be duration neutral to reduce sensitivity to changes in interest rates. 
* be dollar neutral to be self-funded with very low net duration.

## <span style="color:red">1.2</span>

Use the formula seen in class:

$$n_j = -n_i\frac{D_{\$,i}}{D_{\$,j}}$$

Use dollar duration in this formula, based on the **dirty** price.

#### Grading note
Partial credit for using 
* duration instead of dollar duration
* calculating dollar duration via clean price instead of dirty price.

In [3]:
SIZELONG = 100e6

summary.loc['dirty price'] = summary.loc['clean price'] + summary.loc['accrued interest']
summary.loc['dollar duration'] = summary.loc['duration'] * summary.loc['dirty price']

keyLong = summary.loc['ytm'].astype('float64').idxmax()
keyShort = summary.loc['ytm'].astype('float64').idxmin()

summary.loc['contracts',keyLong] = SIZELONG / summary.loc['dirty price',keyLong]
summary.loc['contracts',keyShort] = -summary.loc['contracts',keyLong] * (summary.loc['dollar duration',keyLong]/summary.loc['dollar duration',keyShort])
summary.loc['market value'] = summary.loc['contracts'] * summary.loc['dirty price']

out = summary.loc[['dirty price', 'dollar duration', 'contracts','market value']].T
out.style.format('{:,.2f}')

,dirty price,dollar duration,contracts,market value
207391,89.63,479.48,"-1,000,404.43","-89,668,000.15"
204095,113.30,543.48,"882,591.50","100,000,000.00"


## <span style="color:red">1.3</span>

In the **long-term**, there is no market risk.
* If held to maturity, guaranteed to profit according to difference in YTM.

In the **short-term**, there is market risk.
* The YTM could diverge further, causing a short-term loss. 
* This could pressure the trader to unwind due to financing and short-term losses.

## <span style="color:red">1.4</span>

The table below shows the modified duration and approximated percentage change in value of each side of the trade.

* The final row shows the pnl approximation based on the trade size from `1.2` above.
* Note that the shift in the YTM approximated is half the total YTM spread on each side.

#### Formula

From the notes, modified duration is (for semiannually compounding)

$$D\frac{1}{1+y/2}$$

In [4]:
COMPOUNDING = 2

pnl_approx = summary.loc[['ytm','duration']].copy()

ytm_convergence = pnl_approx.loc['ytm',keyLong] - pnl_approx.loc['ytm',keyShort]
pnl_approx.loc['ytm convergence',keyLong] = -ytm_convergence/2
pnl_approx.loc['ytm convergence',keyShort] = ytm_convergence/2

pnl_approx.loc['modified duration'] = pnl_approx.loc['duration'] / (1+pnl_approx.loc['ytm']/COMPOUNDING)
pnl_approx.loc['pct change value'] = -pnl_approx.loc['modified duration'] * pnl_approx.loc['ytm convergence']

pnl_approx.loc['pnl'] = pnl_approx.loc['pct change value'] * out['market value']
pnl_approx = pnl_approx.T

pnl_approx.loc['net','pnl'] = pnl_approx['pnl'].sum()

pnl_approx.style.format({'ytm':'{:.2%}','pct change value':'{:.2%}','pnl':'${:,.2f}'})

,ytm,duration,ytm convergence,modified duration,pct change value,pnl
207391,3.77%,5.349400,0.000553,5.250489,-0.29%,"$260,588.25"
204095,3.88%,4.796700,-0.000553,4.705452,0.26%,"$260,446.76"
net,nan%,nan,nan,nan,nan%,"$521,035.01"


## <span style="color:red">1.5</span>

The lower coupon security will have higher convexity and thus **less accurate** approximation of pnl via duration.

In [5]:
if summary.loc['coupon rate'].idxmin()==keyLong:
    idPos = 'LONG'
else:
    idPos = 'SHORT'
    
display(f'The {idPos} position will have a less accurate duration approximation due to higher convexity.')

'The SHORT position will have a less accurate duration approximation due to higher convexity.'

## <span style="color:red">1.6</span>

Yes, this difference in YTM is consistent with liquidity. The higher YTM is on the instrument that is much older (issued in 1999) and the lower YTM is on the instrument recently issued ("on-the-run").

## <span style="color:red">1.7</span>

In [6]:
from scipy.optimize import fsolve

def price_bond(ytm, T, cpn, cpnfreq=2, face=100, accr_frac=None):
    ytm_n = ytm/cpnfreq
    cpn_n = cpn/cpnfreq
    
    if accr_frac is None:
        accr_frac = (T-round(T))*cpnfreq
    
    N = T * cpnfreq
    price = face * ((cpn_n / ytm_n) * (1-(1+ytm_n)**(-N)) + (1+ytm_n)**(-N)) * (1+ytm_n)**(accr_frac)
    return price

def ytm(price, T, cpn, cpnfreq=2, face=100, accr_frac=None):
    pv_wrapper = lambda y: price - price_bond(y, T, cpn, cpnfreq=cpnfreq, face=face, accr_frac=accr_frac)
    ytm = fsolve(pv_wrapper,.01)
    return ytm

In [7]:
TTM = 5.5
cpns = summary.loc['coupon rate',:].values
prices = [91.02,109.5]

ylds = [ytm(prices[i],TTM,cpns[i])[0] for i in range(2)]
tab = pd.DataFrame(ylds)
tab.columns = ['ytm']
tab.style.format('{:.2%}')

,ytm
0,3.13%
1,3.78%


***

# 2. Equity Analytics

#### Data

This problem requires use of 
* `select_equity_returns.xlsx`

This file has sheets for...
* `info` - names of each stock ticker
* `single-name stocks` - weekly returns on several stocks
* `spy` - weekly returns on SPY

Note the data is **weekly** so any annualizations should use `52` weeks in a year.

### 2.1.

Report a table which shows the following stats for each of the single-name stocks. Be sure to **annualize**.
* mean
* volatility
* mean-vol (Sharpe) ratio

Which stock seems most attractive? Why?

### 2.2.

Report the following non-linear risk statistics for the single-name equities, (no annualization needed.)
* 5th quantile of returns
* maximum drawdown

### 2.3.

Explain conceptually why the "market beta" may be useful in addition to the performance metrics above.

### 2.4.

Calculate the market beta for each of the single-name stocks, with respect to `SPY`.

$$r_{i,t} = \alpha + \beta\, r_{\text{SPY},t} + \epsilon_t$$

### 2.5.

Short-answer conclusions...

* Suppose the S&P 500 goes up 1%. How much do you expect TSLA to move?

* Which stock's return is most explained by the S&P 500?

***

# <span style="color:red">Solution 2</span>

In [8]:
FILEIN = '../data/select_equity_returns.xlsx'
sheet_stocks = 'single-name stocks'
sheet_spy = 'spy'

rets = pd.read_excel(FILEIN, sheet_name=sheet_stocks).set_index('date')
spy = pd.read_excel(FILEIN, sheet_name=sheet_spy).set_index('date')

## <span style="color:red">2.1</span>

In [9]:
stats = pd.concat([rets.mean(),rets.std(),rets.mean()/rets.std()],axis=1).rename(columns={0:'mean',1:'vol',2:'Sharpe'})
ANNUALIZE = 52
stats['mean'] *= ANNUALIZE
stats['vol'] *= np.sqrt(ANNUALIZE)
stats['Sharpe'] *= np.sqrt(ANNUALIZE)
stats

,mean,vol,Sharpe
AAPL,0.319421,0.283883,1.125183
MSFT,0.288087,0.240206,1.199334
AMZN,0.239457,0.310389,0.771474
NVDA,0.650658,0.468096,1.390011
GOOGL,0.193328,0.274217,0.705020
TSLA,0.569728,0.607026,0.938556
XOM,0.124196,0.311613,0.398557


Based solely on this table, we might find `NVDA` most attractive in terms of its return-per-risk ratio.

Of course, volatility is only one metric for risk, and this ratio is just one measure of reward-per-risk.

## <span style="color:red">2.2</span>

In [10]:
def maximumDrawdown(returns):
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max

    max_drawdown = drawdown.min()
    end_date = drawdown.idxmin()
    summary = pd.DataFrame({'Max Drawdown': max_drawdown, 'Bottom': end_date})

    for col in drawdown:
        summary.loc[col,'Peak'] = (rolling_max.loc[:end_date[col],col]).idxmax()
        recovery = (drawdown.loc[end_date[col]:,col])
        try:
            summary.loc[col,'Recover'] = pd.to_datetime(recovery[recovery >= 0].index[0])
        except:
            summary.loc[col,'Recover'] = pd.to_datetime(None)

        summary['Peak'] = pd.to_datetime(summary['Peak'])
        try:
            summary['Duration (to Recover)'] = (summary['Recover'] - summary['Peak'])
        except:
            summary['Duration (to Recover)'] = None
            
        summary = summary[['Max Drawdown','Peak','Bottom','Recover','Duration (to Recover)']]

    return summary    

In [11]:
risk = maximumDrawdown(rets)
risk.insert(0,'5th quantile',rets.quantile(.05).to_frame())
risk[['5th quantile', 'Max Drawdown']].style.format('{:.2%}')

,5th quantile,Max Drawdown
AAPL,-5.23%,-37.21%
MSFT,-4.94%,-29.95%
AMZN,-6.19%,-46.81%
NVDA,-8.38%,-59.23%
GOOGL,-5.57%,-34.83%
TSLA,-12.25%,-68.22%
XOM,-6.17%,-67.14%


## <span style="color:red">2.3</span>

The performance stats above are all **univariate**. They do not tell us anything about how the stocks perform relative to other assets. The market beta gives a comparison for the overall equity market rather than a comparison to one particular stock.

Accordingly, market beta is an important risk metric to understand the asset performance against broad market movements.

## <span style="color:red">2.4</span>

In [12]:
regstats = get_ols_metrics(spy,rets,annualization=ANNUALIZE)[['SPY','r-squared']]
regstats

,SPY,r-squared
AAPL,1.115316,0.483601
MSFT,1.019508,0.564397
AMZN,1.037437,0.350011
NVDA,1.655707,0.391984
GOOGL,1.071942,0.478768
TSLA,1.776825,0.268439
XOM,0.923144,0.274966


## <span style="color:red">2.5</span>

For a `1%` increase in `SPY`, the regression above indicates we should expect a `1.78%` increase in `TSLA`.

Of course, the regression beta is just the **expected** movement. The fairly low **r-squared** tells us that this average effect will not be the typical effect.

Overall, we see from the r-squared stat that `MSFT` is most explained by `SPY`.

***

***

# 3. Short-Answer

#### No Data Needed

These problems do not require any data file. Rather, analyze them conceptually. 

### 1.

Which likely has higher correlation: two S&P 500 stocks (chosen randomly) or two money-market interest rates (chosen randomly)?

### 2.

Name a mechanism by which the Fed influences money-market rates. Cite a method used which they did not widely use 20 years ago.

Describe a second mechanism the Fed uses to impact rates which is less important in recent times.

### 3.

True or False: We expect high correlation in the relationship between leverage on an asset and the haircut on the asset for purposes of repo.

### 4.

Will the adjusted returns be higher or lower than the unadjusted returns for a stock which is...
* paying high dividends
* splitting frequently

Explain your answer.

### 5.

If SPY is meant to track the S&P 500 (SPX), then why are their returns so different in this chart?

![](../refs/return_chart_midterm.png)

***

# <span style="color:red">Solution 3</span>

## <span style="color:red">3.1</span>

We expect two money-market rates have much higher correlation than the average pairwise correlation of equities. The former correlation may well be over 90%, whereas the latter is expected to be closer to 60%.

## <span style="color:red">3.2</span>

Recently, the Fed's (arguably) most important method for setting money market rates is the Fed's **repo** facility. This allows them to borrow and lend at a set rate with both banks and non-bank financial firms.

Another medium-term mechanism (fine to list as a recent / present) is setting interest on reserves.

Traditionally, the Fed's influence was primarily via adjusting the demand and supply of reserves. Setting the demand and supply of these reserves allowed them to set the **Fed Funds rate.** This mechanism has become less important in an error where banks have abundant excess reserves, so there is less trading of reserves.

## <span style="color:red">3.3</span>

We learned that a haircut is the amount of capital we must fund. Thus, lower haircuts allow us to fund the position with higher debt and leverage. Accordingly, we'd expect a **negative** correlation between the size of the haircut on an asset and the leverage of the position.

## <span style="color:red">3.4</span>

Adjusted returns will be **higher**. The dividends are part of the return, and they are not shown in the unadjusted price series. Similarly, splits will show up in the unadjusted price series as drops in price. The adjusted series will add in the dividends and ensure the splits do not decrease the price series. Both these things lead the adjusted prices (and derived returns) to be higher.

## <span style="color:red">3.5</span>

SPY tracks SPX, but we learned the ETF (SPY) collects and pays dividends on the underlying stocks, whereas the index itself (SPX) does not. 

***

***